<a href="https://colab.research.google.com/github/Kimminsu-ds/Deep-Learning-NLP-using-Tensorflow/blob/main/03_01_Word_Embedding_Word2Vec(English).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 러닝스푼즈 - Tensorflow를 활용한 딥러닝 자연어처리

In [1]:
import re
from lxml import etree
import urllib.request
import zipfile
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize

In [2]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## 데이터 세팅

In [8]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

targetXML = open("ted_en-20160408.xml", 'r', encoding="UTF8")
target_text = etree.parse(targetXML)

# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.
parse_text = '\n'.join(target_text.xpath("//content/text()"))
print(parse_text[:96])

Here are two reasons companies fail: they only do more of the same, or they only do what's new.



## 전처리

In [10]:
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거
# 해당 코드는 괄호로 구성된 내용을 제거
content_text = re.sub(r'\([%)]*\)', '', parse_text)
print(content_text[:95])

Here are two reasons companies fail: they only do more of the same, or they only do what's new.


In [11]:
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화 수행
sent_text = sent_tokenize(content_text)

In [12]:
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환
normalized_text = []
for string in sent_text:
  tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
  normalized_text.append(tokens)

In [13]:
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행
result = [word_tokenize(sentence) for sentence in normalized_text]

In [14]:
print("총 샘플의 개수: {}".format(len(result)))

총 샘플의 개수: 273649


In [15]:
for line in result[:3]:
  print(line)

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']
['both', 'are', 'necessary', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing']


## Word2Vec 임베딩
[Gensim Word2Vec API](https://radimrehurek.com/gensim/models/word2vec.html)
- size: 워드 벡터의 특징값. 즉, 임베딩 된 벡터의 차원
- window: 컨텍스트 윈도우 크기
- min_count: 단어 최소 빈도수 제한(빈도가 적은 단어들은 학습하지 않는다)
- workers: 학습을 위한 프로세스 수
- sg
  - 0: CBOW
  - 1: Skip-gram

In [16]:
from gensim.models import Word2Vec
model = Word2Vec(sentences = result, size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [17]:
model_result = model.wv.most_similar("man")
print(model_result)

[('woman', 0.8565999865531921), ('guy', 0.8201879262924194), ('lady', 0.7860196828842163), ('girl', 0.767379641532898), ('soldier', 0.7628076076507568), ('boy', 0.7621582746505737), ('gentleman', 0.736214280128479), ('kid', 0.7107130289077759), ('poet', 0.6882884502410889), ('surgeon', 0.6674830913543701)]


In [18]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format("eng_w2v") # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format("eng_w2v") # 모델 로드

In [19]:
model_result = loaded_model.most_similar("man")
print(model_result)

[('woman', 0.8565999865531921), ('guy', 0.8201879262924194), ('lady', 0.7860196828842163), ('girl', 0.767379641532898), ('soldier', 0.7628076076507568), ('boy', 0.7621582746505737), ('gentleman', 0.736214280128479), ('kid', 0.7107130289077759), ('poet', 0.6882884502410889), ('surgeon', 0.6674830913543701)]


## Visualization
- eng_w2v라는 Word2Vec 모델이 이미 존재한다는 가정 하에 아래 커맨드를 수행

In [24]:
%pwd

'/content'

In [25]:
!python -m gensim.scripts.word2vec2tensor --input eng_w2v --output eng_w2v

2021-10-19 12:46:07,984 - word2vec2tensor - INFO - running /usr/local/lib/python3.7/dist-packages/gensim/scripts/word2vec2tensor.py --input eng_w2v --output eng_w2v
2021-10-19 12:46:07,985 - utils_any2vec - INFO - loading projection weights from eng_w2v
2021-10-19 12:46:10,568 - utils_any2vec - INFO - loaded (21662, 100) matrix from eng_w2v
2021-10-19 12:46:12,611 - word2vec2tensor - INFO - 2D tensor file saved to eng_w2v_tensor.tsv
2021-10-19 12:46:12,611 - word2vec2tensor - INFO - Tensor metadata file saved to eng_w2v_metadata.tsv
2021-10-19 12:46:12,614 - word2vec2tensor - INFO - finished running word2vec2tensor.py


### [Embedding Projector](https://projector.tensorflow.org/)
  - 위 Choose file 버튼 클릭
  - eng_w2v_tensor.tsv 파일 업로드
  - 아래 Choose file 버튼 클릭
  - eng_w2v_metadata.tsv 파일 업로드
  - 두 파일을 업로드하면 워드 임베딩 모델이 시각화